In [96]:
import tensorflow as tf
import numpy as np
import sys
import pandas as pd
import csv
import functools
np.set_printoptions(precision=3, suppress=True)

In [39]:
%run cli_parser.ipynb
%run utils.ipynb

In [101]:
sys.argv=['main.py','-p', './params.json']
args = CliArgs(sys.argv[1:])
params = args.get_params()
out_path = args.get_out_path()

In [108]:
LABEL_COLUMN = params.layout["target"]
NUMERIC_FEATURES = params.layout["numeric"] 
CATEGORICAL_FEATURES = params.layout["categorical"]
SELECT_COLUMNS = [LABEL_COLUMN] + NUMERIC_FEATURES + CATEGORICAL_FEATURES
print (SELECT_COLUMNS)
train_file_path = params.train_path

['bandgap_energy_ev', 'number_of_total_atoms', 'percent_atom_al', 'spacegroup']


In [77]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
    return dataset

In [129]:
raw_train_data = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(raw_train_data)

spacegroup          : [206 167 206  12  33]
numeric             : [[80.     0.5  ]
 [30.     0.   ]
 [80.     0.375]
 [20.     0.25 ]
 [80.     0.438]]


In [62]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

In [83]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [104]:
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [109]:
num = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
MEAN = np.array(num.T['mean'])
STD = np.array(num.T['std'])
print (MEAN, STD)

[61.679  0.385] [22.598  0.266]


In [117]:
cat = pd.read_csv(train_file_path)[CATEGORICAL_FEATURES]
CATEGORIES = {}
for feat in CATEGORICAL_FEATURES:
    CATEGORIES[feat] = list(cat[feat].unique())
print (CATEGORIES)

{'spacegroup': [33, 194, 227, 167, 206, 12]}


In [95]:
def normalize_numeric_data(data, mean, std):
    # Center the data
    return (data-mean)/std

In [97]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(2,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fc6bcde62f0>, mean=array([61.679,  0.385]), std=array([22.598,  0.266])))

In [105]:
example_batch, labels_batch = next(iter(packed_train_data)) 
example_batch['numeric']

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[40.   ,  0.375],
       [80.   ,  0.312],
       [40.   ,  0.   ],
       [30.   ,  0.667],
       [80.   ,  0.312]], dtype=float32)>

In [106]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.959, -0.039],
       [ 0.811, -0.275],
       [-0.959, -1.451],
       [-1.402,  1.059],
       [ 0.811, -0.275]], dtype=float32)

In [118]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [120]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 0. 1. 0. 0. 0.]


In [121]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [122]:
print(preprocessing_layer(example_batch).numpy()[0])

[-0.959 -0.039  0.     0.     1.     0.     0.     0.   ]
